# FUNCIÓN DE MODELO DE MACHINE LEARNING DE RECOMENDACIÓN DE PELICULAS 

In [1]:
from fastapi import FastAPI, HTTPException
from fastapi.responses import HTMLResponse
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [4]:
#cargando los archivos de datos necesarios

datasets_final = pd.read_csv('/Users/benjaminzelaya/Desktop/ML_Proyecto_Individual_Henry/Sist. Recomendacion de Peliculas - Machine Learning/movies_data_final.csv')
data_genres = pd.read_csv('/Users/benjaminzelaya/Desktop/ML_Proyecto_Individual_Henry/Sist. Recomendacion de Peliculas - Machine Learning/genres_groupby.csv')
cast_idpel_df = pd.read_csv('/Users/benjaminzelaya/Desktop/ML_Proyecto_Individual_Henry/Sist. Recomendacion de Peliculas - Machine Learning/Cast_idPeliculas_df.csv')

In [5]:
datasets_final.head()

,id,original_language,overview,popularity,release_date,runtime,status,tagline,title,vote_average,...,name_genres,id_genres,name_companies,id_companies,Cod_countries,country_name,budget,revenue,release_year,return
0,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,81.0,Released,NaN,Toy Story,7.7,...,Animation,16.0,Pixar Animation Studios,3.0,US,United States of America,30000000,373554033.0,1995,12.451801
1,8844,en,When siblings Judy and Peter discover an encha...,17.015539,1995-12-15,104.0,Released,Roll the dice and unleash the excitement!,Jumanji,6.9,...,Comedy,35.0,TriStar Pictures,559.0,US,United States of America,65000000,262797249.0,1995,4.043035
2,15602,en,A family wedding reignites the ancient feud be...,11.712900,1995-12-22,101.0,Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,...,Family,10751.0,Teitler Film,2550.0,US,United States of America,0,0.0,1995,0.000000
3,31357,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,1995-12-22,127.0,Released,Friends are the people who let you be yourself...,Waiting to Exhale,6.1,...,Adventure,12.0,Interscope Communications,10201.0,US,United States of America,16000000,81452156.0,1995,5.090760
4,11862,en,Just when George Banks has recovered from his ...,8.387519,1995-02-10,106.0,Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,...,Fantasy,14.0,Warner Bros.,6194.0,US,United States of America,0,76578911.0,1995,0.000000


In [6]:
datasets_final.columns

Index(['id', 'original_language', 'overview', 'popularity', 'release_date',
       'runtime', 'status', 'tagline', 'title', 'vote_average', 'vote_count',
       'id_bellongs_to_collection', 'name_bellongs_to_collection',
       'poster_bellongs_to_collection', 'backdrop_bellongs_to_collection',
       'Cod_languages', 'name_languages', 'name_genres', 'id_genres',
       'name_companies', 'id_companies', 'Cod_countries', 'country_name',
       'budget', 'revenue', 'release_year', 'return'],
      dtype='object')

In [7]:
data_genres.head()

,id,name,id_genres
0,100,'Comedy' 'Crime',"'35,80'"
1,10000,'Comedy' 'Drama',"'35,18'"
2,10001,'Comedy' 'Science Fiction',"'35,878'"
3,100010,'Drama' 'War',"'18,10752'"
4,100017,'Drama','18'


In [8]:
data_genres.rename(columns={'name': 'genero'},inplace=True)

## Crear Nuevo DataSets para prueba de ejercicio recomendacion de peliculas indicando nombre de pelicula utilizando popularity.

* El modelo de recomendación que se está utilizando en este caso se basa en el algoritmo de vecinos más cercanos (K-Nearest Neighbors o K-NN). Es un modelo de aprendizaje automático que se utiliza para encontrar objetos similares en función de las características de esos objetos.

* En el contexto de este sistema de recomendación de películas, el modelo de vecinos más cercanos busca películas similares a una película de consulta en términos de dos características: género y popularidad. Para ello, utiliza la métrica de distancia euclidiana para calcular la similitud entre las películas.


* Se cargan los datos de películas desde un data que concatenará un DataFrame de pandas complento y otro donde solamente usaremos dos columnas "title" y "popularity". Se realiza un preprocesamiento para manejar los valores faltantes (NaN) y transformar las columnas de género en variables ficticias (dummy variables).
crearemos  el modelo de vecinos más cercanos con un número determinado de vecinos y se ajusta a los datos de características.
Búsqueda de vecinos cercanos: Cuando se proporciona una película de consulta, el modelo encuentra las películas más cercanas a esa película en términos de género y popularidad.

* Las películas más similares encontradas son las recomendaciones que se devuelven como resultado, es decir, este modelo se basa en la idea de que si dos películas tienen características similares (en este caso, género y popularidad), es probable que los usuarios que disfruten de una película también disfruten de la otra. El algoritmo de vecinos más cercanos encuentra películas similares basándose en la distancia entre ellas en un espacio de características. Es importante tener en cuenta que este modelo de recomendación se enfoca en la similitud de las características y no en modelos de preferencias de usuarios o análisis de contenido más complejos.

In [9]:
# Agregar las nuevas columnas al dataset movie_genres
movie_ML = pd.concat([data_genres, datasets_final[['title', 'popularity']]], axis=1)

# Mostrar las primeras filas del dataset actualizado
movie_ML.head()

,id,genero,id_genres,title,popularity
0,100,'Comedy' 'Crime',"'35,80'",Toy Story,21.946943
1,10000,'Comedy' 'Drama',"'35,18'",Jumanji,17.015539
2,10001,'Comedy' 'Science Fiction',"'35,878'",Grumpier Old Men,11.712900
3,100010,'Drama' 'War',"'18,10752'",Waiting to Exhale,3.859495
4,100017,'Drama','18',Father of the Bride Part II,8.387519


In [10]:
# Ordenar la tabla según las columnas específicas
movie_ML = movie_ML
columns_order = ['id', 'title', 'genero', 'popularity']
movie_ML = movie_ML[columns_order]

# Redondear la columna "popularity" a dos decimales
movie_ML['popularity'] = movie_ML['popularity'].round(2)

# Eliminar caracteres no deseados de la columna "genero"
movie_ML['genero'] = movie_ML['genero'].str.replace(r'[\'"\s\[\]{}\\]', ' ', regex=True)
# Eliminar las comas de la columna "genero"
movie_ML['genero'] = movie_ML['genero'].str.replace(',', '')

In [11]:
movie_ML.head()

,id,title,genero,popularity
0,100,Toy Story,Comedy Crime,21.95
1,10000,Jumanji,Comedy Drama,17.02
2,10001,Grumpier Old Men,Comedy Science Fiction,11.71
3,100010,Waiting to Exhale,Drama War,3.86
4,100017,Father of the Bride Part II,Drama,8.39


In [25]:
#Elimno los duplicados de columna title:

movie_ML.drop_duplicates(subset=['title'], inplace=True)

In [26]:
movie_data = movie_ML

# Creo el dataframe con las columnas indicadas guardando el mismo dentro la carpeta de Machine Learning
movie_data.to_csv("ML_SistemaRecomendacion_1.csv", index=False)

In [27]:
# Quiero obtener la lista de pelis para tomar nombres y poner de ejemplo:

# Convertir los valores en la columna 'title' a cadenas de texto
movie_data['title']= movie_data['title'].astype(str)  

# Obtener todos los nombres únicos en la columna 'title'
peliculas = movie_data['title'].unique().tolist()
peliculas



['Toy Story',
 'Jumanji',
 'Grumpier Old Men',
 'Waiting to Exhale',
 'Father of the Bride Part II',
 'Heat',
 'Sabrina',
 'Tom and Huck',
 'Sudden Death',
 'GoldenEye',
 'The American President',
 'Dracula: Dead and Loving It',
 'Balto',
 'Nixon',
 'Cutthroat Island',
 'Casino',
 'Sense and Sensibility',
 'Four Rooms',
 'Ace Ventura: When Nature Calls',
 'Money Train',
 'Get Shorty',
 'Copycat',
 'Assassins',
 'Powder',
 'Leaving Las Vegas',
 'Othello',
 'Now and Then',
 'Persuasion',
 'The City of Lost Children',
 'Shanghai Triad',
 'Dangerous Minds',
 'Twelve Monkeys',
 'Wings of Courage',
 'Babe',
 'Carrington',
 'Dead Man Walking',
 'Across the Sea of Time',
 'It Takes Two',
 'Clueless',
 'Cry, the Beloved Country',
 'Richard III',
 'Dead Presidents',
 'Restoration',
 'Mortal Kombat',
 'To Die For',
 'How To Make An American Quilt',
 'Se7en',
 'Pocahontas',
 'When Night Is Falling',
 'The Usual Suspects',
 'Guardian Angel',
 'Mighty Aphrodite',
 'Lamerica',
 'The Big Green',
 'Geo

In [33]:
# AQUI ARMO EL SISTEMA DE RECOMENDACION USANDO MODELO DE MACHINE LEARNING DE VECINOS MAS CERCANOS

import pandas as pd
from sklearn.neighbors import NearestNeighbors

def Pelis_recom(pelicula):
    # archivo CSV con los datos
    ML_DF1 = pd.read_csv('/Users/benjaminzelaya/Desktop/ML_Proyecto_Individual_Henry/Sist. Recomendacion de Peliculas - Machine Learning/ML_SistemaRecomendacion_1.csv')

    # Buscar la película por título en la columna 'title'
    movie = ML_DF1[ML_DF1['title'] == pelicula]

    if len(movie) == 0:
        return "La película no se encuentra en la base de datos."

    # Obtener el género y la popularidad de la película
    movie_genero = movie['genero'].values[0]
    movie_popularity = movie['popularity'].values[0]

    # matriz de características para el modelo de vecinos más cercanos
    features = ML_DF1[['popularity']]
    genres = ML_DF1['genero'].str.get_dummies(sep=' ')
    features = pd.concat([features, genres], axis=1)

    # Manejar valores faltantes (NaN) reemplazándolos por ceros
    features = features.fillna(0)

    # modelo de vecinos más cercanos
    nn_model = NearestNeighbors(n_neighbors=6, metric='euclidean')
    nn_model.fit(features)

    # Encontrar las películas más similares (excluyendo la película de consulta indicada por usuario)
    _, indices = nn_model.kneighbors([[movie_popularity] + [0] * len(genres.columns)], n_neighbors=6)
    similar_movies_indices = indices[0][1:]  # Excluyendo la primera película que es la misma consulta
    Pelis_recom = ML_DF1.iloc[similar_movies_indices]['title']

    # Si la película de consulta está en la lista de recomendaciones, la eliminamos
    if pelicula in Pelis_recom.tolist():
        Pelis_recom = Pelis_recom[Pelis_recom != pelicula]

    return Pelis_recom



In [34]:
# Ejemplo de uso de la función

pelicula = 'Avatar'
peliculas_recomendadas = Pelis_recom(pelicula)
print(f"Películas recomendadas para '{pelicula}':")
print(peliculas_recomendadas)


Películas recomendadas para 'Avatar':
25040    Guardians of the Galaxy Vol. 2
23045                         John Wick
25038                          Deadpool
23140                        Big Hero 6
22433                         Gone Girl
Name: title, dtype: object


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


In [39]:
# Ejemplo de uso de la función
Pelis_recom('Toy Story')


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


39859                             The Lost City of Z
8059                                 The Incredibles
35277    I Spit on Your Grave III: Vengeance is Mine
15637                                   TRON: Legacy
37079                                 Train to Busan
Name: title, dtype: object

## PRUEBA DE SISTEMA DE RECOMENDACION DE PELICULAS POR NOMBRE DE ACTORES

In [50]:
# Agregar las nuevas columnas al dataset movie_genres
movie_ML3= pd.concat([data_genres,cast_idpel_df, datasets_final[['title','popularity','overview']]], axis=1)

pd.set_option('display.max_colwidth', None)

# Mostrar las primeras filas del dataset actualizado
movie_ML3.head(100)

,id,genero,id_genres,id_peliculas,Actor,title,popularity,overview
0,100,'Comedy' 'Crime',"'35,80'",862,Tom Hanks,Toy Story,21.946943,"Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences."
1,10000,'Comedy' 'Drama',"'35,18'",862,Tim Allen,Jumanji,17.015539,"When siblings Judy and Peter discover an enchanted board game that opens the door to a magical world, they unwittingly invite Alan -- an adult who's been trapped inside the game for 26 years -- into their living room. Alan's only hope for freedom is to finish the game, which proves risky as all three find themselves running from giant rhinoceroses, evil monkeys and other terrifying creatures."
2,10001,'Comedy' 'Science Fiction',"'35,878'",862,Don Rickles,Grumpier Old Men,11.712900,"A family wedding reignites the ancient feud between next-door neighbors and fishing buddies John and Max. Meanwhile, a sultry Italian divorcée opens a restaurant at the local bait shop, alarming the locals who worry she'll scare the fish away. But she's less interested in seafood than she is in cooking up a hot time with Max."
3,100010,'Drama' 'War',"'18,10752'",862,Jim Varney,Waiting to Exhale,3.859495,"Cheated on, mistreated and stepped on, the women are holding their breath, waiting for the elusive ""good man"" to break a string of less-than-stellar lovers. Friends and confidants Vannah, Bernie, Glo and Robin talk it all out, determined to find a better way to breathe."
4,100017,'Drama','18',862,Wallace Shawn,Father of the Bride Part II,8.387519,"Just when George Banks has recovered from his daughter's wedding, he receives the news that she's pregnant ... and that George's wife, Nina, is expecting too. He was planning on selling their home, but that's a plan that -- like George -- will have to change with the arrival of both a grandchild and a kid of his own."
...,...,...,...,...,...,...,...,...
95,10050,'Comedy' 'Romance',"'35,10749'",949,Hazelle Goodman,Les Miserables,1.479446,"In France during World War II, a poor and illiterate man, Henri Fortin (Jean-Paul Belmondo), is introduced to Victor Hugo's classic novel Les Misérables and begins to see parallels between the book and his own life."
96,10051,'Comedy' 'Drama' 'Romance',"'35,18,10749'",949,Ray Buktenica,Bed of Roses,10.412194,"The seeds of love are planted when Lisa, a high-powered investment banker, receives flowers from a secret admirer. But when his fairy-tale fantasies clash with her workaholic ways, they soon find out that sometimes, it's harder than it seems for love to conquer all."
97,10052,'Drama','18',949,Max Daniels,Big Bully,2.936443,"A writer returns to his hometown where he faces the childhood nemesis whose life he ultimately ruined, only the bully wants to relive their painful past by torturing him once again."
98,100526,'Drama' 'Fantasy' 'War',"'18,14,10752'",949,Vince Deadrick Jr.,Screamers,12.758848,"(SIRIUS 6B, Year 2078) On a distant mining planet ravaged by a decade of war, scientists have created the perfect weapon: a blade-wielding, self-replicating race of killing devices known as Screamers designed for one purpose only -- to hunt down and destroy all enemy life forms."


In [48]:
movie_ML3.drop('id', axis=1, inplace=True)
# dropeo la columna id creada que no tiene ninguna importancia 

In [51]:
# Filtro filas para armar dataset
movie_ML3.head(1000)

,id,genero,id_genres,id_peliculas,Actor,title,popularity,overview
0,100,'Comedy' 'Crime',"'35,80'",862,Tom Hanks,Toy Story,21.946943,"Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences."
1,10000,'Comedy' 'Drama',"'35,18'",862,Tim Allen,Jumanji,17.015539,"When siblings Judy and Peter discover an enchanted board game that opens the door to a magical world, they unwittingly invite Alan -- an adult who's been trapped inside the game for 26 years -- into their living room. Alan's only hope for freedom is to finish the game, which proves risky as all three find themselves running from giant rhinoceroses, evil monkeys and other terrifying creatures."
2,10001,'Comedy' 'Science Fiction',"'35,878'",862,Don Rickles,Grumpier Old Men,11.712900,"A family wedding reignites the ancient feud between next-door neighbors and fishing buddies John and Max. Meanwhile, a sultry Italian divorcée opens a restaurant at the local bait shop, alarming the locals who worry she'll scare the fish away. But she's less interested in seafood than she is in cooking up a hot time with Max."
3,100010,'Drama' 'War',"'18,10752'",862,Jim Varney,Waiting to Exhale,3.859495,"Cheated on, mistreated and stepped on, the women are holding their breath, waiting for the elusive ""good man"" to break a string of less-than-stellar lovers. Friends and confidants Vannah, Bernie, Glo and Robin talk it all out, determined to find a better way to breathe."
4,100017,'Drama','18',862,Wallace Shawn,Father of the Bride Part II,8.387519,"Just when George Banks has recovered from his daughter's wedding, he receives the news that she's pregnant ... and that George's wife, Nina, is expecting too. He was planning on selling their home, but that's a plan that -- like George -- will have to change with the arrival of both a grandchild and a kid of his own."
...,...,...,...,...,...,...,...,...
995,10531,'Drama' 'Thriller' 'Mystery',"'18,53,9648'",117164,Cynthia Rothrock,The Hunchback of Notre Dame,13.553999,"When Quasi defies the evil Frollo and ventures out to the Festival of Fools, the cruel crowd jeers him. Rescued by fellow outcast the gypsy Esmeralda, Quasi soon finds himself battling to save the people and the city he loves."
996,105325,'Animation' 'Comedy',"'16,35'",117164,Marshall R. Teague,The Cable Guy,7.586577,"When recently single Steven moves into his new apartment, cable guy Chip comes to hook him up -- and doesn't let go. Initially, Chip is just overzealous in his desire to be Steven's pal, but when Steven tries to end the ""friendship,"" Chip shows his dark side. He begins stalking Steven, who's left to fend for himself because no one else can believe Chip's capable of such behavior."
997,10533,'Drama' 'History' 'Romance',"'18,36,10749'",117164,Daniel McVicar,Kingpin,11.633347,"After bowler Roy Munson swindles the wrong crowd and is left with a hook for a hand, he settles into impoverished obscurity. That is, until he uncovers the next big thing: an Amish kid named Ishmael. So, the corrupt and the hopelessly naïve hit the circuit intent on settling an old score with Big Ern."
998,10534,'Action' 'Drama',"'28,18'",117164,Lydie Denier,Eraser,6.919314,"U.S. Marshall John Kruger erases the identities of people enrolled in the Witness Protection Program. His current assignment is to protect Lee Cullen, who's uncovered evidence that the weapons manufacturer she works for has been selling to terrorist groups. When Kruger discovers that there's a corrupt agent within the program, he must guard his own life while trying to protect Lee's."


In [52]:
# armo otroa dataframe que contendra los valores para el sistema de recomendacion 2.

# Guardar el DataFrame en un archivo CSV
sistema2 = 'df_ML_SistemaRecomendacionVect.csv'
movie_ML3.to_csv(sistema2, index=False)

print(f"El DataFrame ha sido guardado en el archivo '{sistema2}'")

El DataFrame ha sido guardado en el archivo 'df_ML_SistemaRecomendacionVect.csv'


In [58]:
# utilizando el data creado anteriormente sesgare mas columna para hacer un data mas pequeño para el sistema de recomendacion:

movie_ML3 = pd.read_csv('df_ML_SistemaRecomendacionVect.csv')

# Obtener las primeras 10000 filas del DataFrame
movie_ML3 = movie_ML3.head(10000)

# Guardar las primeras 10000 filas en un nuevo archivo CSV
movie_ML3.to_csv('df_ML_SistemaRecomendacionVect_final.csv', index=False)


/var/folders/9h/wktndydj5rg6pj7yfx6ws1gm0000gn/T/ipykernel_9338/2651008607.py:3: DtypeWarning: Columns (0,1,2,5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  movie_ML3 = pd.read_csv('df_ML_SistemaRecomendacionVect.csv')


# Prueba  RECOMENDACION POR ACTORES

* LISTA DE ACTORES DEL DATA DE 1000 ACTORES, PARA ESTE EJERCICIO UTILIZO EL DATA LLAMADO DF_MACHINE_LEARNING.CSV

In [47]:
# Quiero obtener la lista de actores para tomar nombres y poner de ejemplo:

# Convertir los valores en la columna 'Actor' a cadenas de texto
df_Machine_learning['Actor'] = movie_ML3['Actor'].astype(str)

# Obtener todos los nombres únicos en la columna 'Actor'
actores_unicos = df_Machine_learning['Actor'].unique().tolist()
actores_unicos


['Tom Hanks',
 'Tim Allen',
 'Don Rickles',
 'Jim Varney',
 'Wallace Shawn',
 'John Ratzenberger',
 'Annie Potts',
 'John Morris',
 'Erik von Detten',
 'Laurie Metcalf',
 'R. Lee Ermey',
 'Sarah Freeman',
 'Penn Jillette',
 'Robin Williams',
 'Jonathan Hyde',
 'Kirsten Dunst',
 'Bradley Pierce',
 'Bonnie Hunt',
 'Bebe Neuwirth',
 'David Alan Grier',
 'Patricia Clarkson',
 'Adam Hann-Byrd',
 'Laura Bell Bundy',
 'James Handy',
 'Gillian Barber',
 'Brandon Obray',
 'Cyrus Thiedeke',
 'Gary Joseph Thorup',
 'Leonard Zola',
 'Lloyd Berry',
 'Malcolm Stewart',
 'Annabel Kershaw',
 'Darryl Henriques',
 'Robyn Driscoll',
 'Peter Bryant',
 'Sarah Gilson',
 'Florica Vlad',
 'June Lion',
 'Brenda Lockmuller',
 'Walter Matthau',
 'Jack Lemmon',
 'Ann-Margret',
 'Sophia Loren',
 'Daryl Hannah',
 'Burgess Meredith',
 'Kevin Pollak',
 'Whitney Houston',
 'Angela Bassett',
 'Loretta Devine',
 'Lela Rochon',
 'Gregory Hines',
 'Dennis Haysbert',
 'Michael Beach',
 'Mykelti Williamson',
 'Lamont Johnso

In [14]:
# AQUI ARMO EL SISTEMA DE RECOMENDACION USANDO MODELO DE MACHINE LEARNING DE VECINOS MAS CERCANOS

import pandas as pd
from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import TfidfVectorizer


def movie_recommendation_artista(Artista: str):

    movie_df = pd.read_csv('/Users/benjaminzelaya/Desktop/ML_Proyecto_Individual_Henry/df_sistema_recomendacion_artistas.csv')

    # películas que tengan al actor especificado en la columna 'Actor'
    movies_with_artista = movie_df[movie_df['Actor'] == Artista]

    if len(movies_with_artista) == 0:
        return "El Artista no se encuentra en la base de datos."

    #  matriz de características para el modelo de vecinos más cercanos
    features = movie_df['genero'].str.get_dummies(sep=' ')

    # modelo de vecinos más cercanos solo con las características de género
    nn_model = NearestNeighbors(n_neighbors=6, metric='euclidean')
    nn_model.fit(features)

    # películas más similares (excluyendo las películas del actor indicado por usuario)
    indices = nn_model.kneighbors(features.iloc[movies_with_artista.index, :])

    # Recomendar películas similares
    movie_recommendation_artista = movie_df.iloc[indices[1][0][1:]]['title']

    return movie_recommendation_artista

In [15]:
# Ejemplo de uso de la función

artista = 'Wallace Shawn'
peliculas_recomendadas = movie_recommendation_artista(artista)
print(f"Películas recomendadas para '{artista}':")
print(peliculas_recomendadas)


Películas recomendadas para 'Wallace Shawn':
3034                 Earthquake
8465                  City Heat
8466             Into the Night
8464    The Ghost & Mr. Chicken
3043         Six Ways to Sunday
Name: title, dtype: object


In [4]:

artista = 'Jim Varney'
peliculas_recomendadas = movie_recommendation_artista(artista)
print(f"Películas recomendadas para '{artista}':")
print(peliculas_recomendadas)

Películas recomendadas para 'Jim Varney':
7208    Too Beautiful for You
4829            Lucky Numbers
4436         Puppet Master II
4205                    Lucas
7579      The State of Things
Name: title, dtype: object


In [5]:
artista = 'Robin Williams'
peliculas_recomendadas = movie_recommendation_artista(artista)
print(f"Películas recomendadas para '{artista}':")
print(peliculas_recomendadas)

Películas recomendadas para 'Robin Williams':
5975                The Last Castle
8286             My Boss's Daughter
9239                     Young Adam
27                      Money Train
13      Dracula: Dead and Loving It
Name: title, dtype: object


In [6]:
movie_recommendation_artista('Tim Allen')

2964             The Color of Money
678     Searching for Bobby Fischer
9745           You're a Big Boy Now
8591                  Promised Land
1302                    Normal Life
Name: title, dtype: object

* Este modelo de Machine Learning se basa en el algoritmo de los vecinos más cercanos (K-Nearest Neighbors, K-NN) para recomendar películas basadas en los actores o actrices. El modelo busca similitudes entre las películas basándose en los actores o actrices que participaron en ellas. 

* Se empezo haciendo la carga del conjunto de datos que contiene información sobre las películas, en este caso utilice un dataframe llamada DF_Machine_Learning.csv, incluyendo los actores o actrices que participaron en ellas.
Para el Preprocesamiento de datos empece limpiando y transformar los datos. Que al principio me tiraba errores por los valores NaN, se manejan los valores faltantes (NaN) en la columna de actores o actrices y se utiliza la técnica TF-IDF (Term Frequency-Inverse Document Frequency) para convertir los nombres de los actores en características numéricas.


* Cuando se ingresa el nombre de un actor o actriz específico, el modelo busca las películas que contienen a ese actor o actriz en el conjunto de datos. Luego, utilizando el algoritmo de vecinos más cercanos, encuentra las películas más similares (excluyendo las películas del propio actor o actriz) y las recomienda como películas relacionadas que podrían ser de interés para el usuario.
Este modelo utiliza la distancia euclidiana en el espacio de características (nombres de actores transformados por TF-IDF) para medir la similitud entre películas. Las películas con actores o actrices similares tendrán una distancia euclidiana más pequeña y, por lo tanto, se consideran más cercanas en términos de similitud de actores o actrices.

* Tambien es posible que el modelo recomiende películas que tengan actores o actrices similares, pero no necesariamente significa que el usuario disfrutará todas las películas recomendadas, ya que siempre la eleccion depende del usuario. La efectividad de las recomendaciones dependerá siempre en gran medida de la calidad y cantidad de datos en el conjunto de datos, así como de la relevancia de los actores para las preferencias del usuario.

### Sistema de recomendacion Count Vectorizer

In [16]:
movie_df_4 = pd.read_csv('/Users/benjaminzelaya/Desktop/SistRecomVect.csv')

In [21]:
movie_df_4.isnull().mean()
# encontre valores NaN que no permite que CountVectorizer pueda manejar directamente valores NaN en el texto de entrada.

id            0.000000
overview      0.002933
popularity    0.000000
title         0.000000
dtype: float64

In [22]:
# Los elimino de la columna
movie_df_4['overview'].fillna('', inplace=True)

In [29]:
movie_df_4.to_csv('SistRecomVect.csv', index=False)


In [30]:
movie_df_4.isnull().mean()


id            0.0
overview      0.0
popularity    0.0
title         0.0
dtype: float64

In [27]:

indices=pd.Series(movie_df_4.index, index=movie_df_4['title'])


def recomendacion_peli_vect(title):
    '''Ingresa un nombre de película y devuelve una lista de películas similares basadas en sus resúmenes'''
    
    # instancia textos en vectores numéricos 
    count = CountVectorizer(stop_words='english') 
    count_matrix = count.fit_transform(movie_df_4['overview']) # utilizamos la columna overviwe de nuestro dataframe movie_df_4., y matriziamos el conteo de cada palabra
    
    # # similitud del coseno entre los vectores
    cosine_sim = cosine_similarity(count_matrix, count_matrix)
    
    title = title.replace(' ', '').lower() 
    
    # índice de la película en el movie_df_4
    if title not in movie_df_4['title'].str.replace(' ', '').str.lower().values: 
        return {'Mensaje': 'Película no encontrada'} 
    
    idx = movie_df_4[movie_df_4['title'].str.replace(' ', '').str.lower() == title].index[0] #  busca el índice correspondiente de la película en el DataFrame movie_df_4
    
    # scores de similitud y los ordena.
    sim_scores = list(enumerate(cosine_sim[idx])) #  scores de similitud
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True) 


    sim_scores = sim_scores[1:6] # para que devuelva solamente 5
    
    # índices de las películas similares (excluyendo la película consultada)
    similar_movie_indices = [i[0] for i in sim_scores if i[0] != idx]
    
    # ista de títulos de películas similares
    similar_movie_titles = movie_df_4['title'].iloc[similar_movie_indices].tolist()
    
    return similar_movie_titles

In [28]:
movie_title = "Toy Story" # asigno el valor con el titulo de la pelicula a la variable movie titble 
similar_movies = recomendacion_peli_vect(movie_title)  # utilizando la llamada a la funcion indico la variable donde esta el titulo, y la varible que indica el data donde estan todas las pelis.
similar_movies # me devolvera el resultado obtenido con los titulos de las pelis similares.

['Toy Story 3',
 'Toy Story 2',
 'The 40 Year Old Virgin',
 'Small Fry',
 'Factory Girl']